In [ ]:
!pip install scikit-learn==1.5.1

: 

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.neighbors import NearestNeighbors




In [82]:
movie = pd.read_csv('/Users/vishnuadithya/Downloads/indian movies.csv')

In [ ]:
movie['Language'].unique()

In [ ]:
movie.columns

In [ ]:
movie.isna()

In [ ]:
movie.isna().sum()

# Testing

# splitting

In [ ]:
movie['Genre'].head()

In [88]:
movie['Genre'] = movie['Genre'].apply(lambda x: x.split(','))

In [89]:
genres_list = sorted(set(genre.strip() for genres in movie['Genre'] for genre in genres))
# for genre in genres_list:
#     movies_df[genre] = movies['genres'].apply(lambda x: int(genre in x))

In [90]:
for genres in genres_list:  
     for genre in genres:
          genre=genre.strip();


In [ ]:
genres_list.pop(0)


In [ ]:
genres_list

In [ ]:
movie.columns

In [94]:
#movie= movie.drop('Genre',axis=1,inplace=True)

In [95]:
movie['Genre'] = movie['Genre'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

# Step 2: Ensure all genres are stripped of any leading or trailing whitespace
movie['Genre'] = movie['Genre'].apply(lambda x: [genre.strip() for genre in x])



# Step 4: Create binary columns for each genre
for genre in genres_list:
    movie[genre] = movie['Genre'].apply(lambda x: int(genre in x))

In [ ]:
movie.head()

**# Features**



In [ ]:
movie_features = movie.drop(columns=['ID', 'Movie Name',  'Genre'])

# Convert all columns to numeric and handle non-numeric values
movie_features = movie_features.apply(pd.to_numeric, errors='coerce')

# Fill missing values with the mean of each column
movie_features.fillna(movie_features.mean(), inplace=True)

# Check for any remaining NaN values
print("Number of NaN values after filling with mean:", movie_features.isna().sum().sum())

# Convert any remaining NaN values to zero
movie_features.fillna(0, inplace=True)

# Check again for NaN values
print("Number of NaN values after converting remaining to zero:", movie_features.isna().sum().sum())

# # Standardize the features
# scaler = StandardScaler()
# movie_features_scaled = scaler.fit_transform(movie_features)



In [ ]:
from sklearn.neighbors import NearestNeighbors

# Drop the original genres column
# df = df.drop(columns=['Genre'])

# Separate the movie names and genre features
movie_names = movie['Movie Name']
# genre_features = df.drop(columns=['Movie Name','ID'])

# Initialize the KNN model
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')

# Fit the model
knn.fit(movie_features)


In [ ]:
# Function to find similar movies
def recommend_movies(movie_name, k=5):
    if movie_name not in movie_names.values:
        return "Movie not found"
    
    movie_index = movie_names[movie_names == movie_name].index[0]
    movie_vector = movie_features.iloc[movie_index].values.reshape(1, -1)
    
    distances, indices = knn.kneighbors(movie_vector, n_neighbors=k+1)
    similar_movies = movie_names.iloc[indices.flatten()[1:]].values
    return similar_movies

# Example usage
input_movie = 'Theri'
print(f"Movies similar to {input_movie}: {recommend_movies(input_movie, k=5)}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer

# Load the data
movie = pd.read_csv('/Users/vishnuadithya/Downloads/indian movies.csv')

def preprocess_data(movie):
  

    # Convert 'Year' to numeric and calculate movie age
    current_year = 2024
    movie['Year'] = pd.to_numeric(movie['Year'], errors='coerce')
    movie['Movie_Age'] = current_year - movie['Year']

    # Clean and convert numeric columns
    numeric_columns = ['Timing', 'Rating(10)', 'Votes', 'Movie_Age']
    for col in numeric_columns:
        movie[col] = pd.to_numeric(movie[col].replace(['-', 'unknown'], np.nan), errors='coerce')
   
    

    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    movie[numeric_columns] = imputer.fit_transform(movie[numeric_columns])

    # Normalize numeric columns
    scaler = MinMaxScaler()
    movie[numeric_columns] = scaler.fit_transform(movie[numeric_columns])

    # One-hot encoding for 'Language'
    movie['Language'] = movie['Language'].fillna('Unknown')
    language_dummies = pd.get_dummies(movie['Language'], prefix='Lang')
    

    
    movie = pd.concat([movie, language_dummies], axis=1)


    # Process 'Genre'
    movie['Genre'] = movie['Genre'].fillna('')
    movie['Genre'] = movie['Genre'].apply(lambda x: [genre.strip() for genre in str(x).split(',')] if pd.notna(x) else [])
    genres_list = sorted(set(genre for genres in movie['Genre'] for genre in genres if genre))
    for genre in genres_list:
        movie[f'Genre_{genre}'] = movie['Genre'].apply(lambda x: int(genre in x))

    # Select features for recommendation
    feature_columns = numeric_columns + list(language_dummies.columns) + [f'Genre_{genre}' for genre in genres_list]
    
   
    
    # Final check for NaN values
    movie_features = movie[feature_columns]
    if movie_features.isnull().values.any():
        print("Warning: NaN values still present in the following columns:")
        print(movie_features.columns[movie_features.isnull().any()].tolist())
        print("Filling remaining NaN values with 0")
        movie_features = movie_features.fillna(0)
    

    
    return movie, feature_columns, movie_features

# Preprocess the data
movie, feature_columns, movie_features = preprocess_data(movie)

# Initialize and fit KNN model
knn = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn.fit(movie_features)

def recommend_movies(movie_name, k=5):
    if movie_name not in movie['Movie Name'].values:
        return "Movie not found"
    
    movie_index = movie[movie['Movie Name'] == movie_name].index[0]
    movie_vector = movie_features.iloc[movie_index].values.reshape(1, -1)
    
    distances, indices = knn.kneighbors(movie_vector, n_neighbors=k+1)
    similar_movies = movie.iloc[indices.flatten()[1:]]['Movie Name'].values
    return similar_movies
movie.info
# Example usage
input_movie = 'Gabbar Singh'
print(f"Movies similar to {input_movie}: {recommend_movies(input_movie, k=5)}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer

# Load the data
movie = pd.read_csv('/Users/vishnuadithya/Downloads/indian movies.csv')

def preprocess_data(movie):
    # Strip leading/trailing spaces from string columns
    movie['Movie Name'] = movie['Movie Name'].str.strip()
    movie['Language'] = movie['Language'].str.strip()

    # Convert 'Year' to numeric and calculate movie age
    current_year = 2024
    movie['Year'] = pd.to_numeric(movie['Year'], errors='coerce')
    movie['Movie_Age'] = current_year - movie['Year']

    # Clean and convert numeric columns
    numeric_columns = ['Timing', 'Rating(10)', 'Votes', 'Movie_Age']
    for col in numeric_columns:
        movie[col] = pd.to_numeric(movie[col].replace(['-', 'unknown'], np.nan), errors='coerce')
   
    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    movie[numeric_columns] = imputer.fit_transform(movie[numeric_columns])

    # Normalize numeric columns
    scaler = MinMaxScaler()
    movie[numeric_columns] = scaler.fit_transform(movie[numeric_columns])

    # One-hot encoding for 'Language'
    movie['Language'] = movie['Language'].fillna('Unknown')
    language_dummies = pd.get_dummies(movie['Language'], prefix='Lang')
    movie = pd.concat([movie, language_dummies], axis=1)

    # Process 'Genre'
    movie['Genre'] = movie['Genre'].fillna('')
    movie['Genre'] = movie['Genre'].apply(lambda x: [genre.strip() for genre in str(x).split(',')] if pd.notna(x) else [])
    genres_list = sorted(set(genre for genres in movie['Genre'] for genre in genres if genre))
    for genre in genres_list:
        movie[f'Genre_{genre}'] = movie['Genre'].apply(lambda x: int(genre in x))

    # Select features for recommendation
    feature_columns = numeric_columns + list(language_dummies.columns) + [f'Genre_{genre}' for genre in genres_list]
    
    # Final check for NaN values
    movie_features = movie[feature_columns]
    if movie_features.isnull().values.any():
        print("Warning: NaN values still present in the following columns:")
        print(movie_features.columns[movie_features.isnull().any()].tolist())
        print("Filling remaining NaN values with 0")
        movie_features = movie_features.fillna(0)
    
    return movie, feature_columns, movie_features

# Preprocess the data
movie, feature_columns, movie_features = preprocess_data(movie)

# Initialize and fit KNN model
knn = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn.fit(movie_features)

def recommend_movies(movie_name, movie_language, k=5):
    # Strip leading/trailing spaces from input
    movie_name = movie_name.strip()
    movie_language = movie_language.strip()

    # Filter movies by name and language
    filtered_movies = movie[(movie['Movie Name'].str.lower() == movie_name.lower()) & 
                            (movie['Language'].str.lower() == movie_language.lower())]
    
    if filtered_movies.empty:
        return "Movie not found"
    
    movie_index = filtered_movies.index[0]
    movie_vector = movie_features.iloc[movie_index].values.reshape(1, -1)
    
    distances, indices = knn.kneighbors(movie_vector, n_neighbors=k+1)
    similar_movies = movie.iloc[indices.flatten()[1:]]['Movie Name'].values
    return similar_movies

# Example usage
input_movie = 'Kaashmora'
input_language = 'tamil'
print(f"Movies similar to {input_movie} ({input_language}): {recommend_movies(input_movie, input_language, k=5)}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer

# Load the data
movie = pd.read_csv('/Users/vishnuadithya/Downloads/indian movies.csv')

def preprocess_data(movie):
    # Strip leading/trailing spaces from string columns
    movie['Movie Name'] = movie['Movie Name'].str.strip()
    movie['Language'] = movie['Language'].str.strip()

    # Convert 'Year' to numeric and calculate movie age
    current_year = 2024
    movie['Year'] = pd.to_numeric(movie['Year'], errors='coerce')
    movie['Movie_Age'] = current_year - movie['Year']

    # Clean and convert numeric columns
    numeric_columns = ['Timing', 'Rating(10)', 'Votes', 'Movie_Age']
    for col in numeric_columns:
        movie[col] = pd.to_numeric(movie[col].replace(['-', 'unknown'], np.nan), errors='coerce')
   
    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    movie[numeric_columns] = imputer.fit_transform(movie[numeric_columns])

    # Normalize numeric columns
    scaler = MinMaxScaler()
    movie[numeric_columns] = scaler.fit_transform(movie[numeric_columns])

    # One-hot encoding for 'Language'
    movie['Language'] = movie['Language'].fillna('Unknown')
    language_dummies = pd.get_dummies(movie['Language'], prefix='Lang')
    movie = pd.concat([movie, language_dummies], axis=1)

    # Process 'Genre'
    movie['Genre'] = movie['Genre'].fillna('')
    movie['Genre'] = movie['Genre'].apply(lambda x: [genre.strip() for genre in str(x).split(',')] if pd.notna(x) else [])
    genres_list = sorted(set(genre for genres in movie['Genre'] for genre in genres if genre))
    for genre in genres_list:
        movie[f'Genre_{genre}'] = movie['Genre'].apply(lambda x: int(genre in x))

    # Select features for recommendation
    feature_columns = numeric_columns + list(language_dummies.columns) + [f'Genre_{genre}' for genre in genres_list]
    
    # Final check for NaN values
    movie_features = movie[feature_columns]
    if movie_features.isnull().values.any():
        print("Warning: NaN values still present in the following columns:")
        print(movie_features.columns[movie_features.isnull().any()].tolist())
        print("Filling remaining NaN values with 0")
        movie_features = movie_features.fillna(0)
    
    return movie, feature_columns, movie_features

# Preprocess the data
movie, feature_columns, movie_features = preprocess_data(movie)

# Initialize and fit KNN model
knn = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn.fit(movie_features)

def recommend_movies(movie_name, movie_language, year_gap=None, k=5):
    # Strip leading/trailing spaces from input
    movie_name = movie_name.strip()
    movie_language = movie_language.strip()

    # Filter movies by name and language
    filtered_movies = movie[(movie['Movie Name'].str.lower() == movie_name.lower()) & 
                            (movie['Language'].str.lower() == movie_language.lower())]
    
    if filtered_movies.empty:
        return "Movie not found"
    
    movie_index = filtered_movies.index[0]
    movie_vector = movie_features.iloc[movie_index].values.reshape(1, -1)
    
    distances, indices = knn.kneighbors(movie_vector, n_neighbors=k+1)
    similar_movies = movie.iloc[indices.flatten()[1:]]

    # Apply year gap filter if specified
    if year_gap:
        movie_year = movie.loc[movie_index, 'Year']
        similar_movies = similar_movies[(similar_movies['Year'] >= movie_year - year_gap) & 
                                        (similar_movies['Year'] <= movie_year + year_gap)]

    return similar_movies['Movie Name'].values

# Example usage
input_movie = 'Eega'
input_language = 'Telugu'
year_gap = 5  # Optional: specify the year gap
print(f"Movies similar to {input_movie} ({input_language}) with a year gap of {year_gap}: {recommend_movies(input_movie, input_language, year_gap, k=5)}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer

# Load the data
movie = pd.read_csv('/Users/vishnuadithya/Downloads/indian movies.csv')

def preprocess_data(movie):
    # Strip leading/trailing spaces from string columns
    movie['Movie Name'] = movie['Movie Name'].str.strip()
    movie['Language'] = movie['Language'].str.strip()

    # Convert 'Year' to numeric and calculate movie age
    current_year = 2024
    movie['Year'] = pd.to_numeric(movie['Year'], errors='coerce')
    movie['Movie_Age'] = current_year - movie['Year']

    # Clean and convert numeric columns
    numeric_columns = ['Timing', 'Rating(10)', 'Votes', 'Movie_Age']
    for col in numeric_columns:
        movie[col] = pd.to_numeric(movie[col].replace(['-', 'unknown'], np.nan), errors='coerce')
   
    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    movie[numeric_columns] = imputer.fit_transform(movie[numeric_columns])

    # Normalize numeric columns
    scaler = MinMaxScaler()
    movie[numeric_columns] = scaler.fit_transform(movie[numeric_columns])

    # One-hot encoding for 'Language'
    movie['Language'] = movie['Language'].fillna('Unknown')
    language_dummies = pd.get_dummies(movie['Language'], prefix='Lang')
    movie = pd.concat([movie, language_dummies], axis=1)

    # Process 'Genre'
    movie['Genre'] = movie['Genre'].fillna('')
    movie['Genre'] = movie['Genre'].apply(lambda x: [genre.strip() for genre in str(x).split(',')] if pd.notna(x) else [])
    genres_list = sorted(set(genre for genres in movie['Genre'] for genre in genres if genre))
    for genre in genres_list:
        movie[f'Genre_{genre}'] = movie['Genre'].apply(lambda x: int(genre in x))

    # Select features for recommendation
    feature_columns = numeric_columns + list(language_dummies.columns) + [f'Genre_{genre}' for genre in genres_list]
    
    # Final check for NaN values
    movie_features = movie[feature_columns]
    if movie_features.isnull().values.any():
        print("Warning: NaN values still present in the following columns:")
        print(movie_features.columns[movie_features.isnull().any()].tolist())
        print("Filling remaining NaN values with 0")
        movie_features = movie_features.fillna(0)
    
    return movie, feature_columns, movie_features

# Preprocess the data
movie, feature_columns, movie_features = preprocess_data(movie)

# Initialize and fit KNN model
knn = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn.fit(movie_features)

def recommend_movies(movie_df, movie_features, knn_model, movie_name, movie_language, year_gap=None, k=5):
    # Strip leading/trailing spaces from input
    movie_name = movie_name.strip()
    movie_language = movie_language.strip()
    
    # Filter movies by name and language
    filtered_movies = movie_df[(movie_df['Movie Name'].str.lower() == movie_name.lower()) &
                               (movie_df['Language'].str.lower() == movie_language.lower())]
    
    if filtered_movies.empty:
        return "Movie not found"
    
    movie_index = filtered_movies.index[0]
    movie_vector = movie_features.iloc[movie_index].values.reshape(1, -1)
    
    if year_gap:
        min_gap, max_gap = map(int, year_gap.split('-'))
        movie_year = movie_df.loc[movie_index, 'Year']
        year_filter = (movie_df['Year'] >= movie_year + min_gap) & (movie_df['Year'] <= movie_year + max_gap)
        
        # Get more neighbors initially to account for year filtering
        n_neighbors = min(len(movie_df) - 1, k * 10)  # Increased multiplier
        distances, indices = knn_model.kneighbors(movie_vector, n_neighbors=n_neighbors + 1)
        
        # Create a DataFrame with distances and apply year filter
        similar_movies = pd.DataFrame({
            'index': indices.flatten()[1:],
            'distance': distances.flatten()[1:]
        })
        similar_movies = similar_movies[year_filter.iloc[similar_movies['index']].values]
        
        # If we still don't have enough recommendations, get more neighbors
        while len(similar_movies) < k and n_neighbors < len(movie_df) - 1:
            n_neighbors = min(len(movie_df) - 1, n_neighbors * 2)
            distances, indices = knn_model.kneighbors(movie_vector, n_neighbors=n_neighbors + 1)
            new_similar = pd.DataFrame({
                'index': indices.flatten()[1:],
                'distance': distances.flatten()[1:]
            })
            new_similar = new_similar[year_filter.iloc[new_similar['index']].values]
            similar_movies = pd.concat([similar_movies, new_similar]).drop_duplicates(subset='index')
        
        # Sort by distance and select top k
        similar_movies = similar_movies.sort_values('distance').head(k)
        recommended_indices = similar_movies['index'].values
    else:
        # If no year gap, just get k neighbors
        distances, indices = knn_model.kneighbors(movie_vector, n_neighbors=k+1)
        recommended_indices = indices.flatten()[1:]
    
    return movie_df.loc[recommended_indices, 'Movie Name'].values

# Example usage
input_movie = 'maanagaram'
input_language = 'Tamil'
year_gap = '1-20'  # Optional: specify the year gap

recommendations = recommend_movies(movie, movie_features, knn, input_movie, input_language, year_gap, k=5)
print(f"Movies similar to {input_movie} ({input_language}) with a year gap of {year_gap}:")
for i, rec_movie in enumerate(recommendations, 1):
    print(f"{i}. {rec_movie}")
